In [ ]:
from IPython.display import clear_output
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import re
import dropbox
import sys
import pandas as pd
import numpy as np
import imageio.v2 as imageio
import matplotlib.pyplot as plt
import cv2
import logging
import datetime
import glob
import json
import scipy
import matplotlib as mpl

from subprocess import call
from tifffile import imwrite
from tqdm import tqdm
from pathlib import Path
from amftrack.util.dbx import (
    upload_folder,
    download,
    read_saved_dropbox_state,
    save_dropbox_state,
    load_dbx,
    get_dropbox_folders_prince,
    get_dropbox_video_folders,
    download_video_folders_drop,
    download_analysis_folders_drop,
)
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer
from amftrack.pipeline.launching.run_super import run_parallel_flows
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plot_data import (
    plot_summary,
    save_raw_data,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.high_mag_analysis import (
    HighmagDataset,
    VideoDataset,
    EdgeDataset,
    index_videos_dropbox_new,
    analysis_run,
)
from amftrack.pipeline.functions.transport_processing.high_mag_videos.kymo_class import (
    KymoVideoAnalysis,
    KymoEdgeAnalysis,
)
from amftrack.util.dbx import (upload
)
%load_ext autoreload
%autoreload 2
%matplotlib widget

from amftrack.pipeline.launching.run_super import (
    run_launcher,
    directory_scratch,
    directory_project,
    directory_project,
    run_parallel_stitch,
    run_parallel_transfer,
)
import os
from amftrack.pipeline.functions.image_processing.experiment_class_surf import (
    Experiment,
    save_graphs,
    load_graphs,
    Edge,
    Node,

)
from amftrack.pipeline.functions.image_processing.experiment_util import (
    get_random_edge,
    distance_point_edge,
    plot_edge,
    plot_edge_cropped,
    find_nearest_edge,
    get_edge_from_node_labels,
    plot_full_image_with_features,
    get_all_edges,
    get_all_nodes,
    find_neighboring_edges,
    reconstruct_image,
    reconstruct_skeletton_from_edges,
    reconstruct_skeletton_unicolor,
    reconstruct_image_from_general,
    plot_full,
    plot_edge_color_value,
)
from amftrack.transport.align_video_network import identify_nodes
logging.basicConfig(stream=sys.stdout, level=logging.debug)
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    update_analysis_info,
    get_analysis_info,
    get_current_folders,
    get_folders_by_plate_id,
)
mpl.rcParams["figure.dpi"] = 100
from amftrack.pipeline.functions.transport_processing.high_mag_videos.register_videos import *
from amftrack.pipeline.functions.transport_processing.high_mag_videos.loading import load_video_dataset
from amftrack.pipeline.functions.transport_processing.high_mag_videos.add_BC import *

from amftrack.pipeline.functions.post_processing.time_hypha import *
from amftrack.pipeline.functions.post_processing.extract_study_zone import (
    load_study_zone,
)

In [ ]:
videos_folder = "/projects/0/einf914/videos/"

analysis_folder = "/projects/0/einf914/analysis_videos/CocoTransport/"
analysis_folder_root = "/projects/0/einf914/analysis_videos/"
directory_targ = '/projects/0/einf914/transport/'


In [ ]:
indexes = {
"20230901_Plate310" : 20,
"20230902_Plate310" : 33,
"20230903_Plate310" : 42,
"20230904_Plate310" : 52,
"20230905_Plate310" : 64,
"20230906_Plate310" : 73,
}
indexes = {
"20230810_Plate441" : 29,
"20230811_Plate441" : 41,
"20230812_Plate441" : 47,
"20230813_Plate441" : 60,
}

In [ ]:
plate_id = "441_20230807"
plate_id_video = "20230812_Plate441"
data_obj = load_video_dataset(plate_id_video, videos_folder, analysis_folder, analysis_folder_root)

In [ ]:
update_plate_info(directory_targ, local=True)
all_folders = get_current_folders(directory_targ, local=True)
folders = all_folders.loc[all_folders["unique_id"] == plate_id]
folders = folders.loc[folders["/Analysis/nx_graph_pruned_labeled.p"] == True]
folders = folders.sort_values(by="datetime")

In [ ]:
plate_id_video = "20230812_Plate441"

folders = folders.sort_values(by="datetime")

exp = Experiment(directory_targ)
i = indexes[plate_id_video]
exp.load(folders.iloc[i : i + 1], suffix="_labeled")
for t in range(exp.ts):
    exp.load_tile_information(t)
    add_betweenness(exp,t)

# load_graphs(exp, directory_targ,indexes = [0])



In [ ]:
phis = [get_derivative(edge,t,fun) for edge in edges]
betweenesses = [edge.betweeness(t) for edge in edges]
fig,ax = plt.subplots()
ax.scatter(np.log10(np.abs(phis)),np.log10(np.abs(betweenesses)),alpha = 0.1)
ax.set_xlim(-8,-1)
ax.set_ylim(-8,-1)

In [ ]:
edges = get_all_edges(exp,t)
betweenesses = [edge.betweeness(t) for edge in edges]
betweenesses = [bet for bet in betweenesses if bet>0]

fig,ax = plt.subplots()
ax.hist(np.log10(betweenesses))

In [ ]:
fun = lambda edge: edge.betweeness(t)
phis = [get_derivative(edge,t,fun) for edge in edges]
phis = [bet for bet in phis if bet>1e-10]
fig,ax = plt.subplots()
ax.hist(np.log10(phis))

In [ ]:
import networkx as nx

G = exp.nx_graph[t]
components = nx.connected_components(G)

# Find the largest connected component
largest_component = max(components, key=len)

# Create a new graph representing the largest connected component
largest_component_graph = G.subgraph(largest_component)
exp.nx_graph[t] = largest_component_graph
exp.save_location = ""

load_study_zone(exp)

In [ ]:
edges = get_all_edges(exp, t)
nodes = get_all_nodes(exp, t)
nodes_source = [
    node
    for node in nodes
    if not is_in_study_zone(node, t, 1000, 100)[1]
    and is_in_study_zone(node, t, 1000, 100)[0]
]
nodes_source = [
    node
    for node in nodes
    if not is_in_ROI_node(node, t)
]
nodes_sink = [
    node
    for node in nodes
    if is_in_ROI_node(node, t) and node.degree(t) == 1
]

In [ ]:
ax = plot_full(exp, t, nodes=nodes_source,downsizing = 10)


In [ ]:
zbzdd

In [ ]:

for edge in vid_obj.edge_objs:
    if "network_begin" in edge.mean_data.keys():
        edge_begin = edge.mean_data["network_begin"]
        edge_end = edge.mean_data["network_end"]    # break
        network_edge = Edge(Node(edge_begin,exp),Node(edge_end,exp),exp) 

In [ ]:
# def get_derivative(edge,t,fun):
# edge = network_edge
fun = lambda edge: edge.betweeness(t)
def get_derivative(edge_d,t,fun):
    edges_begin = edge_d.begin.edges(t)
    edges_begin.remove(edge_d)
    edges_end = edge_d.end.edges(t)
    edges_end.remove(edge_d)
    weight_begin = np.sum([fun(edge) for edge in edges_begin])
    weight_end = np.sum([fun(edge) for edge in edges_end])
    return(weight_end-weight_begin)

In [ ]:
edges_network = []
derivatives = []

coverage_tot = []
for index,vid_obj in enumerate(data_obj.video_objs):
    if vid_obj.dataset['mode']=="BF" and check_hasedges(vid_obj):
        for edge in vid_obj.edge_objs:
            if "network_begin" in edge.mean_data.keys():
                edge_begin = edge.mean_data["network_begin"]
                edge_end = edge.mean_data["network_end"]    # break
                network_edge = Edge(Node(edge_begin,exp),Node(edge_end,exp),exp) 
                if network_edge.is_in(t): 
                    edges_network.append(network_edge)
                    derivatives.append(get_derivative(network_edge,t,fun))

In [ ]:
from amftrack.pipeline.functions.transport_processing.high_mag_videos.plotting import *
downsizing = 5
ax,f = plot_full_video( 
    exp,
    t,
    downsizing=5,
    # points=positions_list,
    video_num=[0],
    edges=edges_network,
    dilation=downsizing,
    # region = region,
    figsize = (18,12),
)
for derivative,edge in zip(derivatives,edges_network):
    pixels = edge.pixel_list(t)
    center_arrow = f(pixels[len(pixels)//2])

    begin_arrow = f(pixels[10])
    end_arrow = f(pixels[-10])
    # begin_arrow = edges_network[0].begin.pos(t)
    # end_arrow = edges_network[0].pixel_list(t)[10]
    vector = (end_arrow-begin_arrow)
    vector = vector/np.linalg.norm(vector)*100/downsizing*(1-2*(derivative>0))
    ax.arrow(center_arrow[1],center_arrow[0],vector[1],vector[0],width = 10)
plt.savefig(f"test_{plate_id_video}_derivative.png", dpi=600)
